# Accessibility metrics calculation

## Import libraries

In [ ]:
import pandana as pdna
import osmnx as ox
import networkx as nx
import rasterio
from rasterstats import zonal_stats, point_query
import pandas as pd
import geopandas as gpd
import h3pandas
from shapely.geometry import box
from matplotlib.patheffects import withStroke
import sys
from shapely import wkb
import os
import libpysal as lps
from sklearn.neighbors import KernelDensity
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
import geoplot
import geoplot.crs as gcrs
from scipy.spatial import cKDTree
from libpysal.weights.distance import get_points_array

In [ ]:
# Administrative boundaries
lake = gpd.read_file('../data/input/lake_geneva.gpkg', engine = 'pyogrio')
cantons = gpd.read_file('../data/input/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.gpkg', engine='pyogrio')
communes = gpd.read_file('../data/input/swissBOUNDARIES3D_1_3_TLM_HOHEITSGEBIET.gpkg', engine='pyogrio')
# Only retain communes that are in the canton of Geneva
communes = communes[communes.KANTONSNUM == 25]

cantons = cantons.to_crs(2056)
communes = communes.to_crs(2056)
communes_4326 = communes.to_crs(4326)
canton_ge = cantons[cantons.NAME=='Genève']

In [ ]:
place = "Canton de Genève, CH"

# Get the geometry of the place
gdf = ox.geocode_to_gdf(place)

# Create a buffer around the geometry
buffered_gdf = gdf.to_crs(epsg=2056).buffer(4000).to_crs(gdf.crs)
buffered_gdf.plot()
# plt.savefig('./results/canton_ge_buffered.png', dpi=80)
buffered_gdf = buffered_gdf.to_crs(4326)
# Get the bounding box of the buffered area
bounds = buffered_gdf.total_bounds
_bbox = box(*bounds)

In [ ]:
# Extract the network using the buffered area
G_buffered = ox.graph_from_polygon(_bbox, network_type='walk')

In [ ]:
# impute speed on all edges missing data
G_buffered = ox.add_edge_speeds(G_buffered)

# calculate travel time (seconds) for all edges
G_buffered = ox.add_edge_travel_times(G_buffered)

### OSM POIs loading

In [ ]:
# Function to properly load a GeoDataFrame from parquet
def load_geodataframe(file_path):
    try:
        # Read parquet file
        df = pd.read_parquet(file_path)
        
        # Check if geometry is in WKB format and convert if needed
        if 'geometry' in df.columns and isinstance(df['geometry'].iloc[0], bytes):
            df['geometry'] = df['geometry'].apply(lambda x: wkb.loads(x) if isinstance(x, bytes) else x)
        
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame(df, crs=4326, geometry='geometry')
        
        print(f"Successfully loaded {file_path}")
        return gdf
    
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

In [ ]:
# Define the categories of POIs
categories = ['physical', 'culture', 'education', 'healthcare', 
              'services', 'transport', 'outdoor', 'supplies', 'restaurant']

# Define the data directory
data_dir = '../data/output/'

# Dictionary to store each category's GeoDataFrame
gdfs = {}

# Load each processed GeoDataFrame
for category in categories:
    file_path = os.path.join(data_dir, f'processed_osm_{category}_pois.parquet')
    gdf = load_geodataframe(file_path)
    if gdf is not None:
        gdfs[category] = gdf

In [ ]:
# Also load the combined dataset if it exists
combined_file_path = os.path.join(data_dir, 'processed_osm_all_pois.parquet')
if os.path.exists(combined_file_path):
    all_pois_gdf = load_geodataframe(combined_file_path)
else:
    all_pois_gdf = None
    print("Combined dataset not found")

# Print summary statistics for each category
print("\nSummary Statistics:")
print("-" * 50)
for category, gdf in gdfs.items():
    print(f"\n{category.capitalize()} POIs:")
    print(f"  Total count: {len(gdf)}")
    print(f"  Subcategories:")
    
    # Count POIs by subcategory
    subcat_counts = gdf['subcategory'].value_counts()
    for subcat, count in subcat_counts.items():
        print(f"    - {subcat}: {count}")

In [ ]:
bbox = gpd.GeoDataFrame(geometry=[_bbox], crs=4326)
bbox_2056 = bbox.to_crs(2056)
bbox_2056 = bbox_2056.geometry.iloc[0]

In [ ]:
# Get the bounding box of the buffered area
bounds_ge = canton_ge.to_crs(2056).total_bounds
_bbox_ge = box(*bounds_ge)

bbox_ge = gpd.GeoDataFrame(geometry=[_bbox_ge], crs=4326)
bbox_ge_2056 = bbox_ge.to_crs(2056)
bbox_ge_2056 = bbox_ge_2056.geometry.iloc[0]

In [ ]:
gdfs_bbox = {}  # Will store the filtered GeoDataFrames

for category in categories:
    print('category')
    gdf = gdfs[category].copy()
    
    # Apply the three operations
    # 1. Filter by bounding box
    gdf_bbox = gdf[gdf.geometry.within(bbox_2056)]
    
    # 2. Add a quantity column
    gdf_bbox['quantity'] = 1
    
    # Store the result
    gdfs_bbox[category] = gdf_bbox
    
    # Print summary
    print(f"{category.capitalize()}: {len(gdf)} total POIs, {len(gdf_bbox)} within bbox")

## Accessibility measures 

In [ ]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_buffered)
gdf_edges = gdf_edges.reset_index()

# Get nearest 20 POIs (fitness center) at max 5km
net=pdna.Network(gdf_nodes["x"], gdf_nodes["y"], gdf_edges["u"], gdf_edges["v"],
                 gdf_edges[["length"]])
net.precompute(6500)

## All categories

In [ ]:
# Calculate proximity metrics for all categories
h3_results = {}
categories = list(gdfs_bbox.keys())

for category in categories:
    print(f"Processing {category}...")
    
    # Skip if empty
    if len(gdfs_bbox[category]) == 0:
        print(f"  No POIs in {category} category. Skipping.")
        continue
    
    # Set POIs for this category
    net.set_pois(
        category=category,
        maxdist=6500,
        maxitems=len(gdfs_bbox[category]),
        x_col=gdfs_bbox[category]['lon'],
        y_col=gdfs_bbox[category]['lat']
    )
    
    # Get distances and convert to minutes
    nearest_20_pois = net.nearest_pois(6500, category, num_pois=20)
    nearest_20_pois_time = nearest_20_pois / 75  # 4.5 km/h (default walking speed) = 75 m/min
    
    # Merge with nodes
    gdf_nearest = pd.merge(gdf_nodes, nearest_20_pois_time, left_index=True, right_index=True)
    
    # Create metrics
    gdf_nearest[f'time_to_5th_{category}'] = gdf_nearest[5]
    gdf_nearest[f'time_to_10th_{category}'] = gdf_nearest[10]
    gdf_nearest[f'time_to_20th_{category}'] = gdf_nearest[20]
    
    # Prepare columns for H3 aggregation
    h3_columns = [f'time_to_5th_{category}', f'time_to_10th_{category}', 
                 f'time_to_20th_{category}', 'geometry']
    
    # Aggregate to H3 cells
    h3_cells = gdf_nearest[h3_columns].h3.geo_to_h3_aggregate(resolution=10, operation='mean')
    h3_results[category] = h3_cells
    
    print(f"  Done with {category}")

In [ ]:
# Merge all H3 results
if h3_results:
    # Start with first category's results
    merged_h3 = h3_results[categories[0]]
    
    # Merge with all other categories
    for category in categories[1:]:
        if category in h3_results:
            merged_h3 = merged_h3.merge(h3_results[category].drop(['geometry'],axis=1), left_index=True, right_index=True, how='outer')
    
    # Calculate 15-minute city score
    for category in categories:
        if f'time_to_20th_{category}' in merged_h3.columns:
            merged_h3[f'score_{category}'] = np.maximum(0, 1 - (merged_h3[f'time_to_20th_{category}'] / 15))
    
    score_cols = [f'score_{category}' for category in categories if f'score_{category}' in merged_h3.columns]
    pt_cols = [f'time_to_20th_{category}' for category in categories if f'time_to_20th_{category}' in merged_h3.columns]
    pt_PA_cols = ['time_to_20th_physical','time_to_20th_outdoor', 'time_to_20th_transport']

    if score_cols:
        merged_h3['overall_15min_city_score'] = merged_h3[score_cols].mean(axis=1)
        merged_h3['overall_15min_city_proximity_time'] = merged_h3[pt_cols].mean(axis=1)
        merged_h3['overall_15min_city_pa_proximity_time'] = merged_h3[pt_PA_cols].mean(axis=1)

    # Save results
    merged_h3.to_parquet('../data/output/h3_accessibility_metrics.parquet')
    print("Saved accessibility metrics to h3_accessibility_metrics.parquet")

## Load Données SITG Population Grand-Genève (200m)

In [ ]:
gdf_sitg_pop_grand_ge = gpd.read_file('../data/input/GML_AGGLO_CARREAU_200/AGGLO_CARREAU_200.gml')

In [ ]:
gdf_sitg_pop_grand_ge_bbox = gdf_sitg_pop_grand_ge[gdf_sitg_pop_grand_ge.geometry.within(bbox_2056)]

### Population reachable in 15min - SITG

In [ ]:
x, y = gdf_sitg_pop_grand_ge_bbox.geometry.to_crs(4326).centroid.x, gdf_sitg_pop_grand_ge_bbox.geometry.to_crs(4326).centroid.y

gdf_sitg_pop_grand_ge_bbox["node_ids"] = net.get_node_ids(x, y)
net.set(gdf_sitg_pop_grand_ge_bbox["node_ids"], variable=gdf_sitg_pop_grand_ge_bbox['POP_TOT_GG_2019'], name="pop_sitg")
# 15min walking at 4.5km/h = 1125m
density15min_sitg = net.aggregate(1125, type="sum", decay="linear", name="pop_sitg")

In [ ]:
gdf_sitg_pop_grand_ge_bbox_centroid = gdf_sitg_pop_grand_ge_bbox.copy()
gdf_sitg_pop_grand_ge_bbox_centroid['geometry'] = gdf_sitg_pop_grand_ge_bbox.geometry.centroid.to_crs(4326)

In [ ]:
h3_cells_population = gdf_sitg_pop_grand_ge_bbox_centroid[['POP_TOT_GG_2019','geometry']].h3.geo_to_h3_aggregate(resolution=10, operation='mean')

In [ ]:
gdf_nodes = pd.merge(gdf_nodes, pd.DataFrame(density15min_sitg, columns = ['15min_population_sitg']), left_index=True, right_index=True)

In [ ]:
net.plot(density15min_sitg,
         fig_kwargs={'figsize': [8, 8]},
         plot_kwargs={'cmap': 'BrBG', 's': 8, 'edgecolor': 'none'})

In [ ]:
h3_cells_population_access = gdf_nodes[['15min_population_sitg','geometry']].h3.geo_to_h3_aggregate(resolution=10, operation='mean')

In [ ]:
h3_cells_population_access['perc_15min_population_sitg'] = (h3_cells_population_access['15min_population_sitg']/gdf_sitg_pop_grand_ge_bbox['POP_TOT_GG_2019'].sum())*100

In [ ]:
h3_cells_population = h3_cells_population.reset_index()

In [ ]:
_h3_cells_population_access = pd.merge(h3_cells_population_access, h3_cells_population[['POP_TOT_GG_2019']], left_index=True, right_index=True, how = 'left')

In [ ]:
_h3_cells_population_access[_h3_cells_population_access.POP_TOT_GG_2019.isnull() == False].POP_TOT_GG_2019.sum()

In [ ]:
h3_cells_population_access.plot('15min_population_sitg', legend=True)

In [ ]:
merged_h3_pop = pd.merge(merged_h3, h3_cells_population_access.drop('geometry',axis=1), left_index = True, right_index=True)

## Dual access

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize

In [ ]:
# Create custom colormap
colors_list = ['darkblue', 'lightblue', '#f7f7f7', '#f4a582', 'darkred']
custom_cmap = LinearSegmentedColormap.from_list('custom', colors_list)

# Create normalization that caps values at 30
norm = Normalize(vmin=0, vmax=30)

In [ ]:
# Restrict accessibility measures to the canton of geneva
merged_h3_ge = merged_h3_pop[merged_h3_pop.within(canton_ge.to_crs(4326).geometry.unary_union)]

In [ ]:
# Save results
merged_h3_ge.to_parquet('../data/output/h3_accessibility_metrics_ge_final.parquet')
print("Saved accessibility metrics to h3_accessibility_metrics_ge_final.parquet")

In [ ]:
# Read results
merged_h3_ge = gpd.read_parquet('../data/h3_accessibility_metrics_ge_final.parquet')
print("Read accessibility metrics to h3_accessibility_metrics_ge_final.parquet")

### Proximity time maps

In [ ]:
def plot_accessibility(merged_h3, col, category, communes, lake, canton, custom_cmap, norm,
                       output_dir='../results/Accessibility maps/', show_pois=False, pois_data=None):
    """
    Create accessibility map for a specific category.
    
    Parameters:
    -----------
    merged_h3 : GeoDataFrame
        H3 cells with accessibility metrics
    category : str
        Category name (e.g., 'physical', 'education')
    communes, lake, canton : GeoDataFrame
        Background map features
    custom_cmap, norm : matplotlib colormap and normalization
    output_dir : str
        Directory to save the output files
    show_pois : bool
        Whether to show the POIs on the map
    pois_data : GeoDataFrame
        POIs data to plot (if show_pois is True)
    """
    import matplotlib.pyplot as plt
    from matplotlib.patheffects import withStroke
    import os
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Plot H3 cells with accessibility metric
    column = f'{col}{category}'
    merged_h3.plot(column, cmap=custom_cmap, norm=norm, zorder=5,
                  legend_kwds={'label': f'Proximity time - {category.capitalize()} [min]',
                              'shrink': 0.3},
                  legend=True, ax=ax)
    
    # Add commune labels
    for x, y, label in zip(communes.geometry.centroid.x, communes.geometry.centroid.y, communes['NAME']):
        ax.text(x, y, label, fontsize=5, ha='right', va='bottom',
                path_effects=[withStroke(linewidth=1, foreground='white')], zorder=8)
    
    # Plot POIs if requested
    if show_pois and pois_data is not None:
        pois_data.plot(markersize=3, alpha=0.5, color='orange', ax=ax)
    
    # Add background map elements
    lake.plot(color='lightgrey', zorder=3, ax=ax)
    canton.geometry.boundary.plot(color='grey', ax=ax)
    communes.geometry.plot(color='grey', zorder=2, linewidth=0.5, ax=ax)
    
    # Set plot style
    ax.set_axis_off()
    
    # Save figure
    os.makedirs(output_dir, exist_ok=True)
    plt.savefig(f'{output_dir}/{col}{category}.png', dpi=360, bbox_inches='tight')
    plt.close()
    
    print(f"Plot for {category} saved to {output_dir}/map_20th_{category}_2025.png")

In [ ]:
for category in categories:
    if f'time_to_20th_{category}' in merged_h3_ge.columns:
        plot_accessibility(
            merged_h3_ge,  # Your H3 data in 4326 projection
            'time_to_20th_',
            category,
            communes_4326,
            lake.to_crs(4326),
            canton_ge.to_crs(4326),
            custom_cmap,
            norm
        )

## Combined accessibility

In [ ]:
plot_accessibility(
            merged_h3_ge,  # Your H3 data in 4326 projection
            'overall_15min_city_proximity_time',
            '',
            communes_4326,
            lake.to_crs(4326),
            canton_ge.to_crs(4326),
            custom_cmap,
            norm
        )

In [ ]:
plot_accessibility(
            merged_h3_ge,  # Your H3 data in 4326 projection
            'overall_15min_city_pa_proximity_time',
            '',
            communes_4326,
            lake.to_crs(4326),
            canton_ge.to_crs(4326),
            custom_cmap,
            norm
        )

In [ ]:
gdf_1200_final_urban = pd.read_csv('../data/confidential/gdf_final_1200_urban.csv')
gdf_1200_final_urban = gpd.GeoDataFrame(gdf_1200_final_urban,crs=2056, geometry = gpd.points_from_xy(gdf_1200_final_urban.E, gdf_1200_final_urban.N) )

In [ ]:
# ax = test_ge_wpop.plot('proximity_time_all', cmap=custom_cmap, norm=norm, zorder=5, legend_kwds = {'label':'Proximity time - All [min]','shrink':0.3}, figsize=(12,12), legend=True)
ax = gdf_1200_final_urban.to_crs(4326).plot('overall_15min_city_proximity_time', alpha=0.6, zorder=6, markersize = 3, cmap=custom_cmap, norm=norm, legend_kwds = {'label':'Proximity time - All [min]','shrink':0.3}, figsize=(12,12), legend=True)
for x, y, label in zip(communes_4326.geometry.centroid.x, communes_4326.geometry.centroid.y, communes_4326['NAME']):
    ax.text(x, y, label, fontsize=5, ha='right', va='bottom',
            path_effects=[withStroke(linewidth=3, foreground='white')], zorder=8)
# ax.set_title('A', loc = 'left', size= 16)
ax.set_axis_off()  # Hide axes
# outdoor_pois.plot(markersize = 3, alpha = 0.5, color='orange',ax=ax)
# geoplot.kdeplot(outdoor_pois, n_levels=10, alpha=0.6, fill=False, cmap='Reds', ax=ax)
lake.to_crs(4326).plot(color='lightgrey', zorder=3, ax=ax)
canton_ge.to_crs(4326).geometry.boundary.plot(color='grey', ax=ax)
communes.to_crs(4326).geometry.boundary.plot(color='lightgrey',zorder=1, linewidth=1, ax=ax)
# communes_urban.dissolve('DATUM_ERST').to_crs(4326).geometry.plot(color='grey',zorder=2, linewidth=0.5, ax=ax)
plt.savefig('../results/map_20th_all_with_busparticipants_test.png', dpi=360, bbox_inches='tight')